# In this notebook, I will calculate the swap based on VNIBOR rate using QuantLib library

In [41]:
# Import QuantLib library
from QuantLib import *

### Data used:
A plain vanilla USD swap with 1 Billion VND notional and 5-year maturity. Let the fixed leg pay 8% coupon semiannually, and the floating leg pay VNIBOR+55bps quarterly

In [42]:
reference_date = Date(18,3,2019)
Settings.instance().evaluationDate = reference_date

The easiest case which does not involve the construction yield curve is that we assume the flat term structure. The code is:

In [43]:
rf = 0.05
vnibor = 0.07
day_count = Actual365Fixed()

discount_curve = YieldTermStructureHandle(FlatForward(reference_date, rf, day_count))
vnibor_curve = YieldTermStructureHandle(FlatForward(reference_date, vnibor, day_count))

# Assuming state bank of Vietnam has the similar settlement rules as LIBOR.
vnibor3M_index = USDLibor(Period(3, Months), vnibor_curve)

### Create Calendar, Settlement and Maturity Date

- `QuantLib` does not offer specific class for Vietnam calendar so I will TARGET class to create the calendar
- `effective_date` is the first day of the contract, start 5 business days from now
- `maturity_date` is the final day of the contract, start 5 years from now.
- `Schedule` is created accordingly based on the information below

In [44]:
calendar = TARGET()
effective_date  = calendar.advance(reference_date, 5, Days)
maturity_date = calendar.advance(effective_date, 5, Years)

fixed_tenor = Period(6, Months)
fixed_schedule = Schedule(effective_date, maturity_date, 
                         fixed_tenor, calendar, 
                         ModifiedFollowing, ModifiedFollowing,
                         DateGeneration.Forward, False)

float_tenor = Period(3, Months)
float_schedule = Schedule(effective_date, maturity_date, 
                         float_tenor, calendar,
                         ModifiedFollowing, ModifiedFollowing,
                         DateGeneration.Forward, False)

### Necessary inputs
- Create swap instrument object using `VanillaSwap`

In [45]:
# 1 billion notional
notional = 1000000000
# fixed rate 0.08
fixed_rate = 0.08
fixed_daycount = Actual360()

# spread for swap over 
float_spread = 0.0055
float_daycount = Actual360()

ir_swap = VanillaSwap(VanillaSwap.Payer, notional, 
                      fixed_schedule, fixed_rate, fixed_daycount, 
                      float_schedule, vnibor3M_index, float_spread, float_daycount)

- pass in the swap engine `DiscountingSwapEngine`

In [46]:
swap_engine = DiscountingSwapEngine(discount_curve)
ir_swap.setPricingEngine(swap_engine)

### Final result for swap valuation:

In [47]:
## value for fixed leg
for index, cf in enumerate(ir_swap.leg(0)):
    print("%2d %-30s %10.2f"%(index+1, cf.date(), float(cf.amount())))

 1 September 25th, 2019           40888888.89
 2 March 25th, 2020               40444444.44
 3 September 25th, 2020           40888888.89
 4 March 25th, 2021               40222222.22
 5 September 27th, 2021           41333333.33
 6 March 25th, 2022               39777777.78
 7 September 26th, 2022           41111111.11
 8 March 27th, 2023               40444444.44
 9 September 25th, 2023           40444444.44
10 March 25th, 2024               40444444.44


In [48]:
## value of floating leg
for index, cf in enumerate(ir_swap.leg(1)):
    print("%2d %-30s %10.2f"%(index+1, cf.date(), float(cf.amount())))

 1 June 25th, 2019                19205963.13
 2 September 25th, 2019           19205963.13
 3 December 27th, 2019            19416454.22
 4 March 25th, 2020               18574714.42
 5 June 25th, 2020                19205963.13
 6 September 25th, 2020           19205963.13
 7 December 28th, 2020            19628732.44
 8 March 25th, 2021               18152451.30
 9 June 25th, 2021                19205963.13
10 September 27th, 2021           19626982.76
11 December 27th, 2021            18998896.07
12 March 25th, 2022               18361100.16
13 June 27th, 2022                19626982.76
14 September 26th, 2022           18995509.47
15 December 27th, 2022            19207675.14
16 March 27th, 2023               18783419.08
17 June 26th, 2023                18995509.47
18 September 25th, 2023           18995509.47
19 December 27th, 2023            19416454.22
20 March 25th, 2024               18574714.42


In [50]:
# NPV for fixed-rate payer, floating-rate receiver.
print(ir_swap.NPV())
print(ir_swap.fairSpread())
print(ir_swap.fairRate())
print(ir_swap.fixedLegBPS())
print(ir_swap.floatingLegBPS())

-19388137.154498816
0.009849625613727285
0.07562299174263383
-442954.0913446994
445742.665605758
